In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from t_nachine.strategies import ExtremeRSI
from t_nachine.candlesticks import Candle
from t_nachine.patterns import BullBearPattern
from t_nachine.strategies.utils import get_candles
from t_nachine.optimization.ml.utils import load
from t_nachine.constants import TRADES_ATTRIBUTES


from finta import TA as ta

from tqdm import tqdm
import os 
import pandas as pd
import numpy as np
import yfinance as yf

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
clf = load("logs/model_thresh_70.pkl")

In [10]:
def get_data_from_yahoo(symbol, start_date=None, end_date=None):
    return yf.download(symbol, 
                       start=start_date, 
                       end=end_date, 
                       progress=False)

In [11]:
RSI_THRESH = 0.1
PRED_THRESH = 0.7
def buy_signal(candle0: Candle, candle1: Candle, rsi: float) -> bool:
        """
        buy signal

        Args:
            candle0 (Candle): Today Candle 
            candle1 ([type]): Yesterday Candle
            rsi: yesteradays RSI

        Returns:
            bool: is buy signal triggered
        """

        rsi_below_10 = rsi < RSI_THRESH
        is_bull_bear = BullBearPattern(candle=candle0, pre_candle=candle1)
        return all([rsi_below_10, is_bull_bear])

In [12]:
from datetime import date, timedelta
today = date.today()
start_date = today - timedelta(days=30)
start_date = start_date.strftime("%Y-%m-%d")

In [34]:
symbols = os.listdir('../archive/yahoo/')

for symbol in tqdm(symbols):
    
    df = get_data_from_yahoo(symbol=symbol.split('.')[0], start_date=start_date)
    df.reset_index(inplace=True)
    df['RSI'] = ta.RSI(df, 2) / 100
    if len(df) > 3:
        # pattern happend yesterday we wait one day to decide if we trade or not
        rsi = df.iloc[-3]["RSI"]
        candle0 = Candle(df.iloc[-2].Open, df.iloc[-2].High, df.iloc[-2].Low, df.iloc[-2].Close)
        candle1 = Candle(df.iloc[-3].Open, df.iloc[-3].High, df.iloc[-3].Low, df.iloc[-3].Close)

        if buy_signal(candle0, candle1, rsi):
            print(f"buy trigged for symbol: {symbol}")
            df['VWAP'] = ta.VWAP(df)
            df['ATR'] = ta.ATR(df)
            df['ADX'] = ta.ADX(df) / 100
            df['WILLIAMS'] = ta.WILLIAMS(df) / 100
            df = pd.concat([df, ta.BBANDS(df), ta.VORTEX(df), ta.PIVOT_FIB(df)], axis=1)   

            # fillna
            to_fill_with = df.dropna().iloc[0]
            df.fillna(to_fill_with, inplace=True)

            # normalized bbands 
            for bb in ['VWAP', 'BB_LOWER', 'BB_MIDDLE', 'BB_UPPER', 'pivot', 's1', 's2', 's3', 's4', 'r1', 'r2', 'r3', 'r4']:
                df[bb] /= df.High

            proba = round(clf.predict_proba(df.iloc[-1][TRADES_ATTRIBUTES].values.reshape(1, -1))[:, 1][0], 3)
            pred = proba > PRED_THRESH

            print(f"Model saying is a {pred} signal with a probability of {proba}")
            print("----------------------------------------------------")
            
            with open("logs/extreme_rsi_screener.txt", "a+") as f:
                new_line = "date is: " + today.strftime("%Y-%m-%d") + ", " + f"buy trigged for symbol: {symbol}" + ", " + f"Model saying is a {pred} signal with a probability of {proba}"
                f.seek(0)
                if len(f.read(100)) > 0:
                    f.write("\n")
                f.write(new_line)

with open("logs/extreme_rsi_screener.txt", "a+") as f:
    f.seek(0)
    if len(f.read(100)) > 0:
        f.write("\n\n--------------------------------------------------------------------------------------\n\n")

  0%|                                          | 1/4985 [00:00<34:26,  2.41it/s]


1 Failed download:
- ERGF: None


  0%|                                        | 5/4985 [00:04<1:08:49,  1.21it/s]


1 Failed download:
- GAINM: No data found for this date range, symbol may be delisted


  0%|                                          | 6/4985 [00:04<57:29,  1.44it/s]

...

100%|████████████████████████████████████▊| 4968/4985 [1:34:13<00:21,  1.26s/it]

buy trigged for symbol: micr.us.txt
Model saying is a True signal with a probability of 0.97
----------------------------------------------------


100%|████████████████████████████████████▉| 4980/4985 [1:34:27<00:05,  1.13s/it]


1 Failed download:
- CSBK: No data found for this date range, symbol may be delisted


100%|█████████████████████████████████████| 4985/4985 [1:34:33<00:00,  1.14s/it]
